In [4]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",              # phonepe_pulseyour MySQL username
    password="root", # your MySQL password
    database="phonepe_pulse"  # database name
)

cursor = conn.cursor()

# List of tables you want to drop
tables_to_drop = [
    "map_transactions", "map_users", "map_insurance_country", "map_insurance_hover",
    "top_insurance", "top_transactions", "top_users",
    "aggregated_transactions", "aggregated_users", "aggregated_insurance"
]

for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")
    print(f"Deleted {table}")


print("✅ Tables deleted successfully!")

cursor.close()
conn.close()


Deleted map_transactions
Deleted map_users
Deleted map_insurance_country
Deleted map_insurance_hover
Deleted top_insurance
Deleted top_transactions
Deleted top_users
Deleted aggregated_transactions
Deleted aggregated_users
Deleted aggregated_insurance
✅ Tables deleted successfully!


In [5]:
import mysql.connector

db_config = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "phonepe_pulse"
}

conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

query = [
    """CREATE TABLE aggregated_transactions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        category VARCHAR(50),         -- recharge, p2p, merchant, etc.
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE aggregated_insurance (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE aggregated_users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        registered_users BIGINT,
        app_opens BIGINT,
        brand VARCHAR(50),
        brand_count BIGINT,
        brand_percentage DECIMAL(12,9)
    );""",
    
    """CREATE TABLE map_transactions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        district VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE map_users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        registeredUsers BIGINT,
        app_opens BIGINT
    );""",
    
    """CREATE TABLE map_insurance_country (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        district VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE map_insurance_hover (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        district VARCHAR(100),
        lat DOUBLE,
        lng DOUBLE,
        metric BIGINT
    );""",
    
    """CREATE TABLE top_transactions (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        state_dis_pin VARCHAR(20),      -- district/pincode
        state_dis_pin_name VARCHAR(100),     -- district name or pincode
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE top_insurance (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        state_dis_pin VARCHAR(20),      -- district/pincode
        state_dis_pin_name VARCHAR(100),
        transaction_count BIGINT,
        transaction_amount BIGINT
    );""",
    
    """CREATE TABLE top_users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        year INT,
        quarter INT,
        country_state_level VARCHAR(100),
        location VARCHAR(100),
        state_dis_pin VARCHAR(20),      -- district/pincode
        state_dis_pin_name VARCHAR(100),
        registeredUsers BIGINT
    );"""
]

for q in query:
    cursor.execute(q)

print("✅ Tables created successfully!")

cursor.close()
conn.close()


✅ Tables created successfully!


In [1]:
import mysql.connector
import pandas as pd

def insert_csv_to_mysql(csv_path, table_name, db_config):
    """
    Insert CSV data into a MySQL table safely, handling NULLs correctly.
    """
    try:
        # Connect to MySQL
        conn = mysql.connector.connect(**db_config)
        cursor = conn.cursor()

        # Load CSV
        df = pd.read_csv(csv_path)

        # Convert NaN/NaT to None (MySQL NULL)
        df = df.where(pd.notnull(df), None)

        # Get actual table columns from MySQL
        cursor.execute(f"DESCRIBE {table_name}")
        table_columns = [col[0] for col in cursor.fetchall()]

        # Keep only common columns
        common_cols = [col for col in df.columns if col in table_columns]
        if not common_cols:
            print(f"⚠️ No matching columns for {table_name}. Skipping.")
            return

        df = df[common_cols]

        # Build query dynamically
        columns = ", ".join([f"`{col}`" for col in common_cols])
        placeholders = ", ".join(["%s"] * len(common_cols))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Replace all NaN/NaT with None again (important for executemany)
        data = [tuple(None if pd.isna(x) else x for x in row) for row in df.to_numpy()]

        # Bulk insert
        batch_size = 5000
        for i in range(0, len(data), batch_size):
            cursor.executemany(insert_query, data[i:i+batch_size])

        # Commit
        conn.commit()
        print(f"✅ {len(df)} rows from {csv_path} inserted into {table_name}")

    except Exception as e:
        print(f"❌ Error with {csv_path}: {e}")

    finally:
        cursor.close()
        conn.close()

# ---------- USAGE ----------

db_config = {
    "host": "localhost",
    "user": "root",
    "password": "root",
    "database": "phonepe_pulse"
}

# aggregated_transactions
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_transactions.csv",
    table_name="aggregated_transactions",
    db_config=db_config
)

# aggregated_insurance
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_insurance.csv",
    table_name="aggregated_insurance",
    db_config=db_config
)

# aggregated_users
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_users.csv",
    table_name="aggregated_users",
    db_config=db_config
)
# map_insurance_country
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/map_insurance_country.csv",
    table_name="map_insurance_country",
    db_config=db_config
)

# map_insurance_hover
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/map_insurance_hover.csv",
    table_name="map_insurance_hover",
    db_config=db_config
)

# map_transactions
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/map_transactions.csv",
    table_name="map_transactions",
    db_config=db_config
)

# map_users
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/map_users.csv",
    table_name="map_users",
    db_config=db_config
)

# top_insurance
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/top_insurance.csv",
    table_name="top_insurance",
    db_config=db_config
)

# top_transactions
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/top_transactions.csv",
    table_name="top_transactions",
    db_config=db_config
)

# top_users
insert_csv_to_mysql(
    csv_path="C:/Users/rahul/Data Science/pulse-master/csv_output/top_users.csv",
    table_name="top_users",
    db_config=db_config
)

✅ 5174 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_transactions.csv inserted into aggregated_transactions
✅ 701 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_insurance.csv inserted into aggregated_insurance
✅ 7326 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/aggregated_users.csv inserted into aggregated_users
✅ 14558 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/map_insurance_country.csv inserted into map_insurance_country
✅ 1430349 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/map_insurance_hover.csv inserted into map_insurance_hover
✅ 21612 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/map_transactions.csv inserted into map_transactions
✅ 21616 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/map_users.csv inserted into map_users
✅ 12846 rows from C:/Users/rahul/Data Science/pulse-master/csv_output/top_insurance.csv inserted into top_insurance
✅ 19135 

In [ ]:
transactions = int(results["All_Phonepe_transcations"].values[0])
    total_value = results["Total_payment_value"].values[0]
    avg_tran = results["avg_transaction_value"].values[0]
    category = results["category"].values[0]
    No_of_transcation_st = results["No_of_transcation"].values[0]
    state_dis_pin_name_st =  results["state_dis_pin_name"].values[0]
    Transcation_count_st = results["Transcation_count"].values[0]
    state_dis_pin_name_dis = results["state_dis_pin_name"].values[0]
    Transcation_count_dis = results["Transcation_count"].values[0]
    state_dis_pin_name_pin = results["state_dis_pin_name"].values[0]
    Transcation_count_pin = results["Transcation_count"].values[0]
    country_state_level = results["country_state_level"].values[0]
    location = results["location"].values[0]
    ALL_transcation = results["ALL_transcation"].values[0]
    All_payment_value = results["All_payment_value"].values[0]
    avg_transaction_value = results["avg_transaction_value"].values[0]

In [ ]:
if option == "Transactions":
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown(
            f"<h2 style='color:#00C4B4; font-size:40px;'>{summary_df["All_Phonepe_transcations"].values[0]:,}</h2>",
            unsafe_allow_html=True
    )
    st.caption("All PhonePe transactions (UPI + Cards + Wallets)")
    
    with col2:
        st.markdown(
            f"<h2 style='color:#00C4B4; font-size:40px;'>{summary_df["Total_payment_value"].values[0]}</h2>",
            unsafe_allow_html=True
    )
    st.caption("Total Payment Value")
    
    with col3:
        st.markdown(
            f"<h2 style='color:#00C4B4; font-size:40px;'>{summary_df["avg_transaction_value"].values[0]}</h2>",
            unsafe_allow_html=True
    )
    st.caption("Avg. Transaction Value")

In [ ]:
st.write(len(dataframes))
for idx, df in enumerate(dataframes, start=1):
    st.subheader(f"📊 DataFrame {idx}")
    st.write(df)

In [ ]:
st.write("Sample Data:", df.head())